# Spark Streaming with Stroom

### How to process JSON formatted Kafka messages containing event-logging events.

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which data from `EventGen` application has been ingested and indexed in the manner described in `stroom-analytic-demo`.

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col
from IPython.display import display
import time,os

#### Schema Discovery
It is necessary to specify the structure of the JSON data arriving on the topic.  This structure can be determined at runtime.

As the same format of data is also available via an indexed search using the `stroom-spark-datasource`, one way to determine the JSON schema is by interrogating the data held in the `Sample Index` Stroom index.

The specified pipeline is a Stroom Search Extraction Pipeline that uses the stroom:json XSLT function to create a JSON representation of the entire event.  This field is called "Json" by default but the name of the field that contains the JSON representation can (optionally) be changed with the parameter jsonField.

In this manner, all data is returned as a single JSON structure within the field **json**

In [2]:
schemaDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',
        index='32dfd401-ee11-49b9-84c9-88c3d3f68dc2',pipeline='13143179-b494-4146-ac4b-9a6010cada89')

In [3]:
json_schema = spark.read.json(schemaDf.rdd.map(lambda row: row.json)).schema

json_schema

StructType(List(StructField(EventDetail,StructType(List(StructField(Authenticate,StructType(List(StructField(Action,StringType,true),StructField(Outcome,StructType(List(StructField(Permitted,StringType,true),StructField(Reason,StringType,true),StructField(Success,StringType,true))),true),StructField(User,StructType(List(StructField(Id,StringType,true))),true))),true),StructField(Process,StructType(List(StructField(Action,StringType,true),StructField(Command,StringType,true),StructField(Type,StringType,true))),true),StructField(TypeId,StringType,true))),true),StructField(EventId,StringType,true),StructField(EventSource,StructType(List(StructField(Client,StructType(List(StructField(HostName,StringType,true))),true),StructField(Device,StructType(List(StructField(HostName,StringType,true))),true),StructField(Generator,StringType,true),StructField(System,StructType(List(StructField(Environment,StringType,true),StructField(Name,StringType,true))),true),StructField(User,StructType(List(Struct

#### Working with JSON

Now it is possible to use the JSON structure to more usefully parse the events received from the topic.

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc, 5)

kafkaProps = {'application.id': 'UEBA-DEMO',\
              'auto.offset.reset': 'smallest',\
              'bootstrap.servers': 'localhost:9092',\
              'metadata.broker.list': 'localhost:9092'}


topic = 'ANALYTIC-DEMO-UEBA'

kvs = KafkaUtils.createDirectStream(ssc,[topic],kafkaProps)
kvs.map(lambda x: x[1])

kvs.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2019-11-20 16:12:25
-------------------------------------------
('User60', '[{"StreamId":"4830","EventId":"5099","EventTime":{"TimeCreated":"2019-11-08T12:00:15.000Z"},"EventSource":{"System":{"Name":"VPN","Environment":"DEMO"},"Generator":"VPN Log","Device":{"HostName":"Central VPN"},"Client":{"HostName":"Device309"}},"EventDetail":{"TypeId":"Login","Authenticate":{"Action":"Logon","User":{"Id":"User60"}}}}]')
('User44', '[{"StreamId":"4830","EventId":"5100","EventTime":{"TimeCreated":"2019-11-08T12:02:36.000Z"},"EventSource":{"System":{"Name":"VPN","Environment":"DEMO"},"Generator":"VPN Log","Device":{"HostName":"Central VPN"},"Client":{"HostName":"Device19"}},"EventDetail":{"TypeId":"Login","Authenticate":{"Action":"Logon","User":{"Id":"User44"}}}}]')
('User56', '[{"StreamId":"4830","EventId":"5101","EventTime":{"TimeCreated":"2019-11-08T12:03:46.000Z"},"EventSource":{"System":{"Name":"VPN","Environment":"DEMO"},"Generator":"VPN Log"

KeyboardInterrupt: 